In [1]:
pip install snowflake-connector-python pandas


     -------------------------------------- 910.4/910.4 kB 4.1 MB/s eta 0:00:00
     -------------------------------------- 105.0/105.0 kB 5.9 MB/s eta 0:00:00
     -------------------------------------- 181.5/181.5 kB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 2.9/2.9 MB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 56.9/56.9 kB ? eta 0:00:00
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
     -------------------------------------- 117.6/117.6 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import snowflake.connector
import pandas as pd


In [6]:
snowflake_account = 'lz05490.central-india.azure'
snowflake_user = 'mithun'
snowflake_password = 'Mithun123!!'
snowflake_database = 'TIMESHEET'
snowflake_schema = 'TIMESHEET_SCHEMA'

In [8]:
import snowflake.connector
import pandas as pd
import uuid  # For generating unique table name



# Generate a unique temporary table name
snowflake_table = f'temp_table_{uuid.uuid4().hex}'  # Generates a random UUID hex string

# Snowflake connection
conn = snowflake.connector.connect(
    user=snowflake_user,
    password=snowflake_password,
    account=snowflake_account,
    database=snowflake_database,
    schema=snowflake_schema
)

# CSV file path using raw string literal
csv_file_path = 'projects.csv'

# Read CSV file into Pandas DataFrame
df = pd.read_csv(csv_file_path)

# Create Snowflake cursor
cursor = conn.cursor()

# Create Snowflake table dynamically based on DataFrame columns
create_table_query = f'CREATE OR REPLACE TABLE {snowflake_table} ({", ".join([f"{col} STRING" for col in df.columns])})'
cursor.execute(create_table_query)

# Upload DataFrame to Snowflake table using cursor.executemany()
insert_query = f'INSERT INTO {snowflake_table} VALUES ({", ".join(["?"] * len(df.columns))})'
cursor.executemany(insert_query, df.values.tolist())

# Commit changes
conn.commit()

# Close Snowflake connection and cursor
cursor.close()
conn.close()

# Print the temporary table name for reference
print(f'Temporary table name: {snowflake_table}')


TypeError: not all arguments converted during string formatting